# Add Box Label on image by using YOLOV5 Format 

#### Convert Yolov5 Format to (x1, y1) (x2, y2) 

#### Convert Yolov5 polygon Format to (x1, y1) (x2, y2)


In [29]:
from glob import glob
import cv2
import numpy as np
import matplotlib.pyplot as plt

def convert_y2xy(l, im_w, im_h):
    print(np.array(l.split(' '), np.float32))
    c, xc, yc, w,h = np.array(l.split(' '), np.float32)
    c, x1,y1, x2,y2  = np.array([c, (xc-(w/2))*im_w, (yc-(h/2))*im_h, (xc+(w/2))*im_w, (yc+(h/2))*im_h], np.int32)
    return c, (x1,y1), (x2,y2)

def conv_poloygn_to_box(l, im_w,im_h):
    l = np.array(l.split(' '), np.float32)
    c, xy = l[0], l[1:]
    xs = np.array([xy[i] for i in range(0,len(xy), 2)])
    ys = np.array([xy[i] for i in range(1,len(xy), 2)])

    xmin,ymin = xs.min(), ys.min()
    xmax,ymax = xs.max(), ys.max()

    c, x1,y1, x2,y2  = np.array([c, xmin*im_w,ymin*im_h , xmax*im_w,ymax*im_h], np.int32)
    return c, (x1,y1), (x2,y2)
    

def show_img_yolov5_format(img_name='name_of_img.jpg', path_folder='dataset/test'):
    img_path = path_folder+'/images/'+img_name
    label_path = path_folder+'/labels/'+img_name[:-4]+'.txt'
    img = plt.imread(img_path)
    im_h, im_w = img.shape[:-1]
    with open(label_path, 'r') as f :
        labels = f.readlines()
        f.close()
    for l in labels:
        c, pt1, pt2 = conv_poloygn_to_box(l, im_h, im_w)
        print(c, pt1, pt2)
        cv2.rectangle(img, pt1, pt2, (255,0,0), 5)
    plt.figure(figsize=(10,5))
    plt.imshow(img)
    plt.show()

    return img

# Count number of Instance of Labels

In [1]:
from glob import glob

def count_label_instances(data_path = 'dataset_path', project='train', all = True):
    if all == True:
        l_path = glob(data_path + '/*/labels/*.txt')
    else:
        l_path = glob(data_path + f'/{project}/labels/*.txt')
    l_index = {}
    for l in l_path:
        with open(l, 'r') as f:
            lines = f.readlines()
            f.close()
        for line in lines:
            obj_no = int(line.split(' ')[0])

            try:
                l_index[obj_no] += 1
            except:
                l_index[obj_no] = 1
    
    keys = list(l_index.keys())
    values = list(l_index.values())
    sorted_value_index = np.argsort(values)
    sorted_dict = {keys[i]: values[i] for i in sorted_value_index[::-1]}
    return sorted_dict
            

# Removing unnecessary label from YOLO format

In [ ]:
import os
from glob import glob

def remove_unlisted_label(selected_labels, data_path = 'dataset_path', project='train', all = True, add=0):

    if all == True:
        labels = glob(data_path+'/*/labels/*')
    else:
        labels = glob(data_path + f'/{project}/labels/*.txt')

    
    count_rm = 0
    count_new = 0
    for i, label in enumerate(labels):
        print('we in label no. : ', i, ' $$$ path ', label, ' $$$',  end= ' & ')
        lines, new_lines = [], []
        with open(label, 'r') as f:
            lines = f.readlines()
            f.close()
        for line in lines:
            obj_no = int(line.split(' ')[0])
            if obj_no in selected_labels:
                spilt_line = line.split(' ')
                spilt_line[0] = str(selected_labels.index(int(spilt_line[0])) + add)
                new_lines.append(' '.join(spilt_line))
    
        if len(new_lines) != 0:
            with open(label, 'w') as f:
                f.writelines(new_lines)
                f.close()
            count_new +=1
            print(' we added this label')
        else:
            path_split = label.split('/')
            im_ph = path_split[:4] + ['images'] + [path_split[-1][:-3] + 'jpg']
            im_ph = '/'.join(im_ph)

            os.remove(label)
            os.remove(im_ph)
            count_rm +=1
            print(' we remove this label')
    
    print('no. img that removed: ', count_rm, '\nno. img that added', count_new)


# Creating daya.yaml file from labels' name list

In [ ]:
def create_data_yaml_file(data_folder='Dataset_folder', out_file_ph ='data.yaml', seleced_names = ['l1, l2']):
    data_yaml = f"""# Mohamed Mosa
    path: ../{data_folder}  # dataset root dir
    train: train/images  # train images 
    val: valid/images  # val images 
    test: test/images # test images (optional)

    names:"""
    for i, label in enumerate(seleced_names):
        data_yaml = data_yaml + f'\n  {i}: {label}'
    with open(out_file_ph,'w') as f:
        f.write(data_yaml)
        f.close()